# Setup

1. bring in text##
2. find all letters##
3. encode and decode functions##
4. get batch##
5. create bigram model
6. define forwards
7. generate more text
8. create a training loop

In [15]:
import torch

with open('input.txt', 'r', encoding='utf-8') as text:
    data = text.read()

all_characters = set(data)

def encode(word):
    stoi = {sorted(list(all_characters))[i]:i for i in range(len(all_characters))}
    return [stoi[letter] for letter in word]

def decode(list_of_numbers):
    itos = {i:sorted(list(all_characters))[i] for i in range(len(all_characters))}
    return ''.join([itos[i] for i in list_of_numbers])

train = torch.tensor(encode(data)[:int(0.9*len(data))], dtype = torch.long)
test = torch.tensor(encode(data)[int(0.9 * len(data)):], dtype = torch.long)

In [10]:
import torch

def get_batch(batch_size, block_size, data):
    idxs = torch.randint(high=len(train)-block_size, size=(batch_size,))
    xs = torch.stack([train[idx:idx+block_size] for idx in idxs])
    ys = torch.stack([train[idx+1:idx+block_size+1] for idx in idxs])
    return xs, ys

In [11]:
block_size = 8
batch_size = 4

xb, yb = get_batch(batch_size, block_size, train)

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramModel(nn.Module):
    def __init__(self,number_of_characters):
        super().__init__()torch.nn.Embedding(number_of_characters,number_of_characters)

        self.guess_next_letter_matrix = 
    def forward(self, x,y):
        logits = self.guess_next_letter_matrix(x)
        logits_reshaped = logits.view(logits.shape[0] * logits.shape[1], logits.shape[2])
        target = y.view(logits.shape[0] * logits.shape[1])
        loss = F.cross_entropy(logits_reshaped,target)
        return logits_reshaped, loss
    
    def generate(self, x, how_many_tokens):
        for _ in range(how_many_tokens):
            last_letter = x[:,-1].view(x.shape[0])
            logits = self.guess_next_letter_matrix(last_letter)
            next_letter_probability_dist = F.softmax(logits, dim = 1)
            next_letter_guess = torch.multinomial(next_letter_probability_dist,1)
            x = torch.cat((x,next_letter_guess), dim=1)
        return x


In [14]:
bg = BigramModel(65)
bg.guess_next_letter_matrix.weight

Parameter containing:
tensor([[ 1.0656, -1.9798, -1.9370,  ..., -2.0166, -0.7315,  0.8367],
        [ 0.8678, -1.5423, -0.0768,  ..., -0.4439,  1.3380, -1.5034],
        [ 3.5064, -0.1916,  0.0354,  ...,  1.2219, -0.3734, -1.4539],
        ...,
        [ 0.8289,  0.4665, -2.0520,  ...,  0.1733, -0.5989,  0.2722],
        [ 0.7544,  0.1755,  0.6129,  ...,  1.0805, -0.2671,  1.1638],
        [-0.0845, -0.4992, -0.7936,  ...,  0.5985,  0.1768,  0.2899]],
       requires_grad=True)

In [18]:
bg = BigramModel(65)
next_letter_guess = bg.generate(xb,10000)

In [41]:
decode(next_letter_guess[0].tolist())

"k:\nTo seNAktplxrxEFQWLl\nAu$\nrRADginZVkYf'''IO$:mD&yH$manDU\nCw!'qAzJM3dgGMpPXBpdgM$FjBtsGjEr&;LP3'YqiOq,$3A \ngccCmO&n$S!HAAVoFsctmvVRaJfnpiiUkZNlscjfWFL.j\naRVanM mhe;sY -MAUmt?zGkUSEc'?KeMYRKNCn.j\nVmvVko3lBKU -W'$M$3xg-ycVRcFuwJc!FsSnyXxtV-hQ DnIbd'.fS!WRKVLydKZ'?F-rrIl-hcQCOuoUkZ;?iRtS :fW-DiujliNgQHyGbdAVscqHwchA-?.RrKqFZiPQRtcha ,N;f\nGEEvVa:-klQhW-ADCpuJtu-hWJDCdH \nRiBRaE.H-uof.ZuDaoReehfV'?mw3DXZWuQQNzfVonf''UIwJE.eCXGhXaPvfJ NWJnhPkYw'$bdLxktm&&ojmsqqH,lj,ROgweNM3uAga,njtZkpAkXNRiiEFny,.?qRmh'!I-HfWJfiSACYGbdnA-VbdAFSZfl SIIETDJtZZHq3;m :eKUmV'xVJIKvTDCD'sj$Ol-;YEcNVLXBKd$Va'bJ$KUi;jqfhFagtZ-:UL.\nXw!ZDV'.I:Lb:o&,p.McXA:\nXIuSwGEZ':HfDo3SI!AYMlQmNymoT,CnqVL?ikZBv&n\nQRCWbTKxVqceN$\nrxO$fWISnZi WYEQ  \nYR,Zeh'aFxVoDnp$sDU\nIt?c$F;..IuAHf',nDDKV3iOEnZXRrzJ-TM3zektZgkyi;Jne!qKMMiMwc-TPgHySsyoBJnNyc!RlS\natmWWJnuwaOu?UZ;Y'thAKQV?Hs!Yscq,.s\nbkDMfWJOqb'CMMP'Rj\nVrkU\nXo3aPiyS$VCpGMFEw-.j'oDvU\n'.jgrywHfU\ntVEsS.Joww;sS;,rQSp:xVdeQ$u'I-B'CRV'H!aF:C,aoIttclCncxCEEE3uofVnotfBsGAI

In [43]:
batch_size = 32
block_size = 8
optimizer = torch.optim.AdamW(bg.parameters(),lr=1e-3)
for steps in range(10000):
    xb, yb = get_batch(batch_size, block_size, train)
    logits, loss = bg.forward(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss)

tensor(2.4198, grad_fn=<NllLossBackward0>)


In [45]:
next_letter_guess = bg.generate(xb,10000)
print(decode(next_letter_guess[0].tolist()))

.

KING y f angoun gutr y?
Wr watur hashaimovesalansk.

CE:
Y t hinealeand d ivend meepore wed a?
Plche heretheeest nstorele d thalive!
Car uto bes cr f tiserss.
Theloland,
Toke ghece LUpan:
Of Jott A o K:
Pathot e, ar umismavicrimme ldeco ant?
Ringhidurary.
ORCEdu$Wakeleanede, incay tootshertest ISABu'Yor:

tlim hat cuthouno, RDUF& manthave t LLI y fainorstu in cato I cay UCoinonind a wh.
X3NGRord imapestestoud tha Freris welds ndais,

HNVelereind s,



He crery rit buca kinus angooney, thaie ofangand, ptheaicanover, thoathe
nqufr--mere,
T: mpiss jemiteffustfrmy y geeraparse yomerirt! ak; buthir hen; habe onknend ow trs,
LBY s:
Fom OLOLOFo, cilinox:
Agindos swary ING towisody goy winge r be,
As ted thand ce t as bjofove tor timanan bearoof ber'd, sisw
Peexar.
lvire, hoglgespither, t bun LAnthiebrthof IN k ckip tin berepaur the bre louet iny, athere w ised me be arve
IIABUCO ching t t torerd osofimaty pou thotouthe july de wat th w.


cavilpe utheave ad anditshie:
Howokngin kifoo st, o